# Exploratory Data Analysis

This notebook contains EDA on the initial dataset to determine any initial correlations, determine any erroneous data points and propose an initial model fitting subject to external analysis and variables.

## THIS NOTEBOOK IS INCOMPLETE. FURTHER STEPS:
 1. Consider any transformations of the landsat data that might aid in increasing correlation
 2. See if terraclimate data can be extracted and mapped against the validation and test set
 3. And if 2 is possible, repeat multivariate analysis for that


## Preparation

Before we start our EDA, just re-importing our packages and re-defining our data frames.

### Requirements and Packages

In [ ]:
!pip install uv
!uv pip install  -r requirements.txt 

In [ ]:
## import packages

import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Data manipulation and analysis
import numpy as np
import pandas as pd
from IPython.display import display

# Multi-dimensional arrays and datasets (e.g., NetCDF, Zarr)
import xarray as xr

# Geospatial raster data handling with CRS support
import rioxarray as rxr

# Raster operations and spatial windowing
import rasterio
from rasterio.windows import Window

# Feature preprocessing and data splitting
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.spatial import cKDTree

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os 


#NEW PACKAGES
import planetary_computer 
import dask 
from scipy import stats
from datetime import datetime
from dask.distributed import Client

### Loading in data

We need to get this to work - terradata to map to lat/long?

This code is all avaliable in benchmark_model_notebook_snowflake.ipynt

In [ ]:
Water_Quality_df = pd.read_csv("water_quality_training_dataset.csv")
landsat_train_features = pd.read_csv("landsat_features_training_all_bands.csv")
Terraclimate_df = pd.read_csv("terraclimate_features_training.csv")

##landsat_train_features['NDMI'] = landsat_train_features['NDMI'].astype(float)
##landsat_train_features['MNDWI'] = landsat_train_features['MNDWI'].astype(float)
landsat_train_features['Sample Date'] = pd.to_datetime(landsat_train_features['Sample Date'],  format='%d-%m-%Y')

def combine_two_datasets(dataset1,dataset2,dataset3):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    
    data = pd.concat([dataset1,dataset2,dataset3], axis=1)
    data = data.loc[:, ~data.columns.duplicated()]
    return data

wq_data = combine_two_datasets(Water_Quality_df, landsat_train_features, Terraclimate_df)
wq_data['Sample Date'] = pd.to_datetime(wq_data['Sample Date'],  format='%d-%m-%Y')
wq_data = wq_data.drop(columns=["Unnamed: 0"])

for column in wq_data.columns:
    wq_data[wq_data[column] == -9999][column] = np.nan 


display(wq_data.head(5))

Water_Quality_df_validate = pd.read_csv("submission_template.csv")
landsat_train_validate = pd.read_csv("landsat_features_validation.csv")
Terraclimate_df_validate = pd.read_csv("terraclimate_features_validation.csv")

landsat_train_validate['NDMI'] = landsat_train_validate['NDMI'].astype(float)
landsat_train_validate['MNDWI'] = landsat_train_validate['MNDWI'].astype(float)
landsat_train_validate['Sample Date'] = pd.to_datetime(landsat_train_validate['Sample Date'],  format='%d-%m-%Y')

wq_data_validation = combine_two_datasets(Water_Quality_df_validate, landsat_train_validate, Terraclimate_df_validate)
wq_data_validation['Sample Date'] = pd.to_datetime(wq_data_validation['Sample Date'],  format='%d-%m-%Y')

## Initial Exploration:

Begin by just observing the Metadata

In [ ]:
wq_data.shape

In [ ]:
wq_data.info()

Approx ~ 1100 null values are in the dataaet. These are pretty common in the training set and also present in the validation and submission datasets - so we cannot ignore

## Single Variate Distributions:

Plotting the histogram all the variables to see their distribution. This is useful to understand the qualities of the dataset, and may help in choosing a algorithm to fit against the data. 

More of note, distribution analysis comes in handy as we need to do some Imputation of missing data values in Landsat features.

In [ ]:
wq_data_EDA = wq_data.drop(columns=['Latitude', 'Longitude'])

wq_data_EDA.describe().T

In [ ]:
numerical_columns = wq_data_EDA.select_dtypes(include=["int64", "float64"]).columns

sns.set_palette("Blues")

plt.figure(figsize=(100, len(numerical_columns) * 5))
for idx, feature in enumerate(numerical_columns, 1):
    plt.subplot(len(numerical_columns), 2, idx)
    sns.histplot(wq_data_EDA[feature], kde=True)
    plt.title(f"{feature} | Skewness: {round(wq_data_EDA[feature].skew(), 2)}")

plt.tight_layout()
plt.show()

Lets ignore Lat and Long as they aren't really that relevant for EDA.

 - Most of our LandSat Predictors are unimodally distributed (dare I say almost normal?) albeit with very large positive skews. Any model we fit may need some checks against positive skewed values as they could be high leverage observations. The only outlier to this is pet, which seems to be normally distributed.

Our dependent variables are distributed very interestingly. All three share the qualities of being multimodal, although Alkalinity could be argued that it is one stretched mode long mode. Important to note: all three response variables have a really long tail. This can really increase SE estimates and have a large impact on model fitting. 

### Transformations and Outlier Handling

We might transform these variables so that we can fit them better. Plotting the Box-Cox transformation gives:

Also replace the response variable in the dataframe with the box cox version.

In [ ]:
Total_Alkalinity_bc, Total_Alkalinity_lambda_opt = stats.boxcox(wq_data_EDA['Total Alkalinity'])
Electrical_Conductance_bc, Electrical_Conductance_lambda_opt = stats.boxcox(wq_data_EDA['Electrical Conductance'])
Dissolved_Reactive_Phosphorus_bc, Dissolved_Reactive_Phosphorus_lambda_opt = stats.boxcox(wq_data_EDA['Dissolved Reactive Phosphorus'])

print(f"Total Alkalinity Optimal Lambda (λ): {Total_Alkalinity_lambda_opt}")
print(f"Electrical Conductance Optimal Lambda (λ): {Electrical_Conductance_lambda_opt}")
print(f"Dissolved Reactive Phosphorus Optimal Lambda (λ): {Dissolved_Reactive_Phosphorus_lambda_opt}")

plt.figure(figsize=(20, 7))

plt.subplot(1, 3, 1)
plt.hist(Total_Alkalinity_bc, bins=30, color='blue', alpha=0.7)
plt.title('Total Alkalinity Transformed Data (Box-Cox)')

plt.subplot(1, 3, 2)
plt.hist(Electrical_Conductance_bc, bins=30, color='green', alpha=0.7)
plt.title('Electrical Conductance Transformed Data (Box-Cox)')

plt.subplot(1, 3, 3)
plt.hist(Dissolved_Reactive_Phosphorus_bc, bins=30, color='green', alpha=0.7)
plt.title('Dissolved Reactive Phosphorus Transformed Data (Box-Cox)')

plt.show()

##wq_data_EDA['Total Alkalinity'] = Total_Alkalinity_bc
##wq_data_EDA['Electrical Conductance'] = Electrical_Conductance_bc
##wq_data_EDA['Dissolved Reactive Phosphorus'] = Dissolved_Reactive_Phosphorus_bc

This could help our analysis

Consider lastly just a check for outlieres with a boxplot on the predictor variables that have high skew. As their leverage will ruin our analysis

In [ ]:
##boxplot = wq_data_EDA.boxplot(column=["Total Alkalinity", "Electrical Conductance", "Dissolved Reactive Phosphorus"])
boxplot = wq_data_EDA.boxplot(column=["nir", "green", "swir16", "swir22"])


The existence of some outliers is concerning, we may also want to conduct some transformation on the predictors. But first lets just see what the validation dataset looks like in terms of these predictors:

In [ ]:
numerical_columns = wq_data_validation.select_dtypes(include=["int64", "float64"]).columns

sns.set_palette("Reds")

plt.figure(figsize=(20, len(numerical_columns) * 2))
for idx, feature in enumerate(numerical_columns, 1):
    plt.subplot(len(numerical_columns), 2, idx)
    sns.histplot(wq_data_validation[feature], kde=True)
    plt.title(f"{feature} | Skewness: {round(wq_data_validation[feature].skew(), 2)}")

plt.tight_layout()
plt.show()

The submission dataset doesn't have this skew, we could be cheeky and remove the 'outliers' from the training set. For nir, green and swir, this maxes out at 20,000 compared to the 60,000 in the training. If you refer to the boxplot above, you'll see it fits the box and whisker plot nicely.

But reducing these nir, green and swir figures will also reduce the NDMI and MNDWI, as they are a ratio of above. Comparing the two histograms of training vs validation data will indicate that the skew in these variables is reduced as well.

We will do some explorations of this in the next section.

## Multi Variate Analysis

NOTE: WE MAY NEED TO SEGMENT OUT THE LANDSAT AND TERRACLIMATE, PENDING ON WHETHER OR NOT WE CAN EXTRACT AND MAP THE TERRACLIMATE

Lets now look at how the variates interact with each other to start looking at the underpinnings of a model.

First a pairwise plot against the transformed predictors to see how all the variables are related.

In [ ]:
sns.set_palette("Blues")

plt.figure(figsize=(25, 25))

sns.pairplot(wq_data_EDA)

plt.suptitle('Pair Plot for DataFrame')
plt.show()

wq_data_EDA_nodate = wq_data_EDA.drop(columns=['Sample Date'])

plt.figure(figsize=(20, 20))

sns.heatmap(wq_data_EDA_nodate.corr(), annot=True, fmt='.2f', cmap='Pastel2', linewidths=2)

plt.title('Correlation Heatmap')
plt.show()

There are a couple things of importance to note here:
 - Predictors nir, green, swir 16 and swir 22 are highly correlated. Fitting more than 1 or 2 may result in an overfit of the model. We can consider some dimension reduction of these predictors as well.
 - Pairwise plots of the three response variables indicate little correlation. However this could be to do with scaling issues and outliers. A transformation to reduce extreme values (or the removal of such) may actually help the correlation to appear!
 - PET is not correlated with any of the landsat data

Further, some of other steps we can take. We can consider the Box-Cox transformation of the response above and then re-consider the pairwise plots:


In [ ]:
wq_data_EDA['Total Alkalinity'] = Total_Alkalinity_bc
wq_data_EDA['Electrical Conductance'] = Electrical_Conductance_bc
wq_data_EDA['Dissolved Reactive Phosphorus'] = Dissolved_Reactive_Phosphorus_bc

wq_data_EDA_nodate = wq_data_EDA.drop(columns=['Sample Date'])

plt.figure(figsize=(25, 25))

sns.pairplot(wq_data_EDA)

plt.suptitle('Pair Plot for DataFrame')
plt.show()

wq_data_EDA_nodate = wq_data_EDA.drop(columns=['Sample Date'])

plt.figure(figsize=(20, 20))

sns.heatmap(wq_data_EDA_nodate.corr(), annot=True, fmt='.2f', cmap='Pastel2', linewidths=2)

plt.title('Correlation Heatmap')
plt.show()

It still doesn't look great. The three response variables are now much more correlated with each other. Seems an approach we can take is try and use one or another of the response variables to predict the others, once we have an estimate.

Otherwise it is still hard to determine any relationship between the predictors and the response. Partly this is because of the skewness in the predictors, we can try one of two things for our EDA.

Firstly lets consider removing some of the outliers. 

### Predictor Outlier and Boundary Analysis

Lets just start by plotting the four main landsat features against themselves. This is the training dataset

In [ ]:
wq_data_EDA_ladnsat_predictors = wq_data_EDA[["nir", "green", "swir16", "swir22"]]

plt.figure(figsize=(25, 25))

sns.pairplot(wq_data_EDA_ladnsat_predictors)

plt.suptitle('Pair Plot for DataFrame')
plt.show()

We note that alot of the ourliers for each nir, green, swir 16, swir 22 are also outliers for the other predictors, so it may just be a select few datapoints we are removing. 

Again for the validation dataset - predictors pairwise gives:

In [ ]:
wq_data_validation_ladnsat_predictors = wq_data_validation[["nir", "green", "swir16", "swir22"]]

sns.set_palette("Reds")

plt.figure(figsize=(25, 25))

sns.pairplot(wq_data_validation_ladnsat_predictors)

plt.suptitle('Pair Plot for DataFrame')
plt.show()

Observe a less linear relationship visible in the validation set. Also note the bounds of each predictor in the validation. 

It makes sense to train the data on a set that indicative of the overall validation set. Lets observe what happens when we do som restricting.

### Predictor Outlier Removal

Lets remove some of the outlier values from the training set and re-check the pairwise plots

In [ ]:
wq_data_EDA_nodate_filtered = wq_data_EDA_nodate[wq_data_EDA_nodate["green"] < 20000]
wq_data_EDA_nodate_filtered = wq_data_EDA_nodate_filtered[wq_data_EDA_nodate_filtered["swir16"] < 20000]
wq_data_EDA_nodate_filtered = wq_data_EDA_nodate_filtered[wq_data_EDA_nodate_filtered["swir22"] < 18000]
##wq_data_EDA_nodate_filtered = wq_data_EDA_nodate_filtered[wq_data_EDA_nodate_filtered["nir"] < 25000]


sns.set_palette("Blues")

plt.figure(figsize=(25, 25))

sns.pairplot(wq_data_EDA_nodate_filtered, kind='reg')

plt.suptitle('Pair Plot for DataFrame')
plt.show()

plt.figure(figsize=(20, 20))

sns.heatmap(wq_data_EDA_nodate_filtered.corr(), annot=True, fmt='.2f', cmap='Pastel2', linewidths=2)

plt.title('Correlation Heatmap')
plt.show()

This still doesn't give us much unfortunately, but it does give us a few things:
 - Correlation between Alkalinity and Electrical Conductance does exist. So if we can fit one, another can also be fitted
  - 'Green' is looking like an indicator variable, with two distinct groups. This can be seen in both the test and training data (<7500, >7500) with a difference in means.
  - PET is strong, perhaps we can fit more terradata as well.
  - At this point, NIR + Green are probably the two others to consider?

### Other things to Consider:
The following can also be considered as part of the analysis:
- Maybe can try transformations of the variables (e.g. functions, indicator variables)
- Lag or Lead of variables could help as well
- Time series analysis

## Analysis of Null values

Begin by just comparing where the null values take place, both spacially and temporally:

 

In [ ]:
wq_data_nonullvalues = wq_data.dropna(how='any',axis=0)
wq_data_nullvalues = wq_data[wq_data["nir"].isnull()]

wq_data_nonullvalues_validation = wq_data_validation[~wq_data_validation["nir"].isnull()]
wq_data_nullvalues_validation = wq_data_validation[wq_data_validation["nir"].isnull()]

plt.scatter(wq_data_nonullvalues["Latitude"], wq_data_nonullvalues["Longitude"] , color='blue', label='Non-Null - Training', marker='.')

plt.scatter(wq_data_nullvalues["Latitude"], wq_data_nullvalues["Longitude"] , color='blue', label='Null - Training', marker='x')

plt.scatter(wq_data_nonullvalues_validation["Latitude"], wq_data_nonullvalues_validation["Longitude"] , color='red', label='Non-Null - Validation', marker='.')

plt.scatter(wq_data_nullvalues_validation["Latitude"], wq_data_nullvalues_validation["Longitude"] , color='red', label='Null - Validation', marker='x')

# Add labels, a title, and a legend
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Assessment of Null Values by Lat and Long')
plt.legend() # Displays the labels we set in plt.scatter()

# Display the plot
plt.show()

# Plotting
plt.figure(figsize=(10, 6))
sns.kdeplot(data=wq_data_nonullvalues['Sample Date'], fill=False, label='Training - No Null', common_norm=False, color = "Blue")
sns.kdeplot(data=wq_data_nullvalues['Sample Date'], fill=False, label='Training - Null', common_norm=False, color = "Blue", ls='--')
sns.kdeplot(data=wq_data_nonullvalues_validation['Sample Date'], fill=False, label='Validation - No Null', common_norm=False, color = "Red")
sns.kdeplot(data=wq_data_nullvalues_validation['Sample Date'], fill=False, label='Validation - Null', common_norm=False, color = "Red", ls='--')
plt.title('Overlayed KDE Plots (Different Sizes)')
plt.legend()
plt.show()

It seems all the nuull values occur in the same place, although their time frames might be different (observe the kernel density plot). Lets confirm exactly how many null value we have exact matches for:

In [ ]:
merged_no_null = pd.concat([wq_data_nonullvalues, wq_data_nonullvalues_validation])
merged_no_null = merged_no_null[["Latitude", "Longitude", "Sample Date"]]

no_null_training_check = pd.merge(wq_data_nullvalues, merged_no_null, on=["Latitude", "Longitude", "Sample Date"], how='inner')
no_null_validation_check = pd.merge(wq_data_nullvalues_validation, merged_no_null, on=["Latitude", "Longitude", "Sample Date"], how='inner')

print(wq_data_nullvalues.shape)
print(no_null_training_check.shape)

print(wq_data_nullvalues_validation.shape)
print(no_null_validation_check.shape)

So it seems while the locations are the same, the dates are different. We will need to impute missing values

The good news is the data seems missing at random and so imputation is possible!

## Next Steps

Null value handling!
File at: